In [ ]:
#Import Python Libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os, sys
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
stocks = pd.read_csv('../../datasets/raw_analyst_ratings.csv')


In [ ]:
print(stocks.shape)

In [ ]:
stocks.isna().sum()